In [ ]:
-- CREATE OR REPLACE FUNCTION python_workloads.profisee_mdm_prd.get_secret(secret_name STRING)
-- RETURNS STRING
-- LANGUAGE PYTHON
-- RUNTIME_VERSION = 3.9
-- HANDLER = 'get_secret_function'
-- EXTERNAL_ACCESS_INTEGRATIONS = (azure_api_access_integration_eis_prd)
-- SECRETS = ('cred1' = python_workloads.profisee_mdm_prd.eis_ingress_prd_secret, 
--            'cred2' = python_workloads.profisee_mdm_prd.eis_engress_connection_string
--           )  -- Add more secrets here
-- AS
-- $$
-- import _snowflake

-- def get_secret_function(secret_name):
--     # Check the secret name and retrieve the corresponding secret
--     secret_value = _snowflake.get_generic_secret_string(secret_name)
--     return secret_value
-- $$;

-- grant all on function python_workloads.profisee_mdm_prd.get_secret(STRING) to role svc_engineer_prd;


In [ ]:
-- SELECT * FROM development.profisee.customer_needs_profisee_matching;

In [ ]:
import requests
import simplejson as json  
from concurrent.futures import ProcessPoolExecutor, as_completed, ThreadPoolExecutor
import pandas as pd
from snowflake.snowpark import Session
from snowflake.snowpark.context import get_active_session
import time
import random
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

session = get_active_session()

def clean_data(data):
    if isinstance(data, dict):
        return {key: clean_data(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [clean_data(item) for item in data]
    elif isinstance(data, float):
        if pd.isna(data) or data in [float('inf'), float('-inf')]:
            return None  
    return data
    
def create_session_with_retries():
    session = requests.Session()
    retries = Retry(
        total=5,  
        backoff_factor=1,  
        status_forcelist=[500, 502, 503, 504],  
    )
    session.mount('https://', HTTPAdapter(max_retries=retries))
    return session
    
def send_post_request(session, data, api_key):
    url = 'https://hleis-mdm-apim-prd.azure-api.net/ingress-consumer/v0/consume'
    headers = {
        'Ocp-Apim-Subscription-Key': api_key,
        'Content-Type': 'application/json'
    }
    clean_payload = clean_data(data)
    max_attempts = 3
    for attempt in range(max_attempts):
        try:
            response = session.post(url, headers=headers, data=json.dumps(clean_payload, use_decimal=True), timeout=10)
            if response.status_code in [200, 201, 202]:
                return response.status_code
            else:
                print(f"Attempt {attempt + 1} failed: Status {response.status_code}", flush=True)
        except requests.ConnectionError as e:
            print(f"ConnectionError on attempt {attempt + 1}: {e}", flush=True)
        time.sleep(random.uniform(0.5, 2))  
    return None  
    
def process_batch(batch_df, api_key, max_workers=0):
    payloads = [{
        "payloads": [{
            "domain": "Account",
            "data": clean_data(batch_df.iloc[i].to_dict())
        }]
    } for i in range(len(batch_df))]
    results = []
    session = create_session_with_retries()  
    with session:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = [executor.submit(send_post_request, session, payload, api_key) for payload in payloads]
            for future in as_completed(futures):
                try:
                    result = future.result(timeout=15)  
                    if result is None:
                        print("API call failed after retries.", flush=True)
                    results.append(result)
                except Exception as e:
                    print(f"An error occurred: {e}", flush=True)
    return results
    
def process_single_batch(start, batch_size, result_df, api_key, max_workers):
    batch_df = result_df[start:start + batch_size]
    print(f"Processing batch from {start} to {start + len(batch_df)}", flush=True)
    batch_results = process_batch(batch_df, api_key, max_workers=max_workers)
    return batch_results, len(batch_df)
    
def main(req=None, batch_size=5000, max_workers=10) -> dict:
    api_secret_query = "SELECT python_workloads.profisee_mdm_prd.get_secret('cred1')"
    api_key = session.sql(api_secret_query).collect()[0][0]
    # query = "SELECT * FROM bi.profisee.customer_needs_profisee_matching"  
    query = "SELECT * FROM bi.profisee.customer_needs_profisee_matching_onetime_catchup"  
    result_df = session.sql(query).to_pandas()
    total_records = len(result_df)
    response_statuses = []
    total_processed = 0
    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(process_single_batch, start, batch_size, result_df, api_key, max_workers)
                   for start in range(0, total_records, batch_size)]
        for future in as_completed(futures):
            try:
                batch_results, processed_count = future.result()
                response_statuses.extend(batch_results)
                total_processed += processed_count
                print(f"Total messages processed: {total_processed}", flush=True)
            except Exception as e:
                print(f"An error occurred during batch processing: {e}", flush=True)
    response_content = {
        "message": "All data sent.",
        "total_records": total_records,
        "status_codes_summary": {status: response_statuses.count(status) for status in set(response_statuses)}
    }
    return response_content
if __name__ == "__main__":
    response_data = main()
    print(f"Final processed total: {response_data['total_records']}", flush=True)
